In [1]:
%pip install -q ultralytics
%pip install -q opencv-python-headless


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import zipfile
from tqdm import tqdm

fname = '/home/jupyter/datasphere/project/train_dataset_train.zip'
path = './'

with zipfile.ZipFile(fname, 'r') as zf:
    for entry in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(entry, path)
        except zipfile.error as e:
            pass


Extracting :  86%|████████▌ | 859/999 [06:39<04:11,  1.79s/it]

In [1]:
import os
from ultralytics import YOLO
import cv2
from tqdm import tqdm

In [4]:
model = YOLO('yolov8n-cls.pt')

100%|██████████| 5.28M/5.28M [00:00<00:00, 19.6MB/s]


In [ ]:
results = model.train(data='detect_shots.yaml', epochs=20, imgsz=640)

In [5]:
model = YOLO('artefacts/shots_detector_run2.pt')

In [ ]:
# Функция для конвертации времени в формате чч:мм:сс в количество секунд
def convert_time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s


train_dir = "/home/jupyter/datasphere/project/train"

folders = os.listdir(train_dir)
folders.sort()

for folder in folders:
    video_files = os.listdir(os.path.join(train_dir, folder))
    for video_file in tqdm(video_files):
        try:
            # Загрузите видео
            video_path = os.path.join(train_dir, folder, video_file)
            cap = cv2.VideoCapture(video_path)

            # Убедитесь, что видео было успешно загружено
            if not cap.isOpened():
                raise Exception("Error opening video file")

            # Получите частоту кадров видео
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Вычислите номера кадров для начального и конечного времени
            start_time = convert_time_to_seconds('00:01:40')
            end_time = convert_time_to_seconds('00:02:30')
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)

            # Установите начальную позицию видео
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            i = 0
            step = 50
            # Читайте видео с начальной до конечной позиции
            while cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame:
                ret, frame = cap.read()
                # Проверьте, успешно ли прочитан кадр
                if not ret:
                    break

                if i % step == 0:
                    # Сохраните кадр
                    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                    frame_name = os.path.join(frames_dir, f'{os.path.split(video_file)[1]}_frame_{frame_number}.jpg')
                    if os.path.exists(frame_name):
                        break
                    cv2.imwrite(frame_name, frame)
                    
                
                i += 1

            # Освободите захват видео и закройте все окна
            cap.release()
        except Exception:
            pass
cv2.destroyAllWindows()